In [1]:
### Cosinus Based (Recommender System KNN)

In [2]:
import pandas as pd

In [3]:
ratings=pd.read_csv('ml-100k/u.data',sep='\t',header=None,names=['user_id','movie_id','rating'],usecols=range(3))

In [4]:
ratings.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [5]:
movies=pd.read_csv('ml-100k/u.item',sep='|',encoding='iso-8859-1',header=None)

In [6]:
movies

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
import numpy as np
movieProperties = ratings.groupby("movie_id").agg({"rating": [np.size, np.mean]})
movieProperties.head()

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

In [8]:
movieNumRatings = pd.DataFrame(movieProperties["rating"]["size"])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x-np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.774914
2,0.223368
3,0.152921
4,0.357388
5,0.146048


In [9]:
movieDict = {}
#burda filmin dosyasini okuyoruz
with open(r'ml-100k/u.item', encoding="iso-8859-1") as f:
    for line in f:
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0]) # movieId'ye aliyoruz, 0. sutuna eslestiriyoruz
        name = fields[1] #ismini buraya aliyoruz
        genres = fields[5:24] # genre'sini 5 ile 24 sutunlara aliyoruz
        genres = list(map(int, genres)) # bunu list yapiyoruz
        movieDict[movieID] = (name, genres, movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))
        # burada filme ayit bir sözlük olusturduk

In [10]:
#bir filmin diger filme uzakligini bulan function
from scipy import spatial
def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB) # cosinus uzakligini buluyoruz, bir genre'nin diger genre'ya
    popularityA = a[2]
    popularityB = b[2]
    popularitiyDistance = abs(popularityA - popularityB) # bir popularity'nin diger popularity'e uzakligini ve mutlak degerini aliyoruz ( abs() )
    return genreDistance + popularitiyDistance

In [13]:
#aradaki mesafeler en kücük olanlari liste olarak istiyoruz
import operator
def getNeighbors(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []            
    for x in range(K):
        neighbors.append(distances[x][0])
        return neighbors
K=30
avgRating = 0
neighbors = getNeighbors(1, K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print( movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))

Liar Liar (1997) 3.156701030927835
